## Cloning the facenet pytorch model from github

In [ ]:
 ! git clone https://github.com/timesler/facenet-pytorch.git facenet_pytorch

# Getting Embeddings

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import numpy as np
import pandas as pd
import torch.nn as nn
import cv2
from facenet_pytorch import MTCNN, InceptionResnetV1
from PIL import Image


device = torch.device('cpu')
print('Running on device: {}'.format(device))

# Define MTCNN module

# Note that, since MTCNN is a collection of neural nets and other code, the
# device must be passed in the following way to enable copying of objects when
# needed internally.
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, prewhiten=True,
    device=device
)

def cos_sim(a, b):
    """Takes 2 vectors a, b and returns the cosine similarity according 
    to the definition of the dot product"""
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

def cos(a,b):
    minx = -1 
    maxx = 1
    return (cos_sim(a,b)- minx)/(maxx-minx)



# Define Inception Resnet V1 module

resnet = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Define a dataset and data loader
dataset = datasets.ImageFolder('data/china')
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
loader = DataLoader(dataset, collate_fn=lambda x: x[0])

# Perfom MTCNN facial detection
aligned = []
names = []
for x, y in loader:
    x_aligned, prob = mtcnn(x, return_prob=True)
    if x_aligned is not None:
        print('Face detected with probability: {:8f}'.format(prob))
        aligned.append(x_aligned)
        names.append(dataset.idx_to_class[y])

# Calculate image embeddings
aligned = torch.stack(aligned).to(device)
embeddings = resnet(aligned).cpu()



# Print distance matrix for classes

cos_sim = nn.CosineSimilarity(dim=-1, eps=1e-6)
for i in range(0,len(names)):
    emb=embeddings[i].unsqueeze(0) 
    dist =cos(embeddings[0],emb)
    
    
dists = [[cos(e1,e2).item() for e2 in embeddings] for e1 in embeddings]
print(pd.DataFrame(dists, columns=names, index=names))



## Face Recognition from Images

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1,extract_face
from PIL import Image,ImageDraw
import torch
import cv2
import torch.nn as nn

def cos_sim(a, b):
    """Takes 2 vectors a, b and returns the cosine similarity according 
    to the definition of the dot product"""
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

def cos(a,b):
    minx = -1 
    maxx = 1
    return (cos_sim(a,b)- minx)/(maxx-minx)



def verify(embedding):
    
    for i,k in enumerate(embeddings):
        for j,l in enumerate(embedding):
            dist =cos(k,l)
    
            
    # Chosen threshold is 0.85  
            if dist > 0.85:
                text=names[i]
                
                cv2.putText(im, text,(boxes[j][0].astype(int),boxes[j][3].astype(int)), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (0,255,0), 2)
                print(text)
            
device = torch.device('cpu')        
resnet = InceptionResnetV1(pretrained='vggface2').eval().to('cpu')
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, prewhiten=True,
    device=device,keep_all=True
)
# Get cropped and prewhitened image tensor
img = Image.open('yang-yang.jpeg')
img_cropped = mtcnn(img)
boxes,prob=mtcnn.detect(img)
img_draw = img.copy()
draw = ImageDraw.Draw(img_draw)

for i, box in enumerate(boxes):
    draw.rectangle(box.tolist())
    extract_face(img, box, save_path='detected_face_{}.png'.format(i))
img_draw.save('annotated_faces.png')
    
# Calculate embedding (unsqueeze to add batch dimension)
im=cv2.imread('annotated_faces.png')
img_embedding = resnet(img_cropped)


#print(img_embedding.size())
cos_sim = nn.CosineSimilarity(dim=-1, eps=1e-6)
verify(img_embedding)
    

cv2.imshow("faces",im)
k=cv2.waitKey(0)
if k==27:
    cv2.destroyAllWindows()
#print(img_embedding.size())
# Or, if using for VGGFace2 classification
##resnet.classify = True
#img_probs = resnet(img_cropped.unsqueeze(0))
#print(img_cropped)
#print(img_probs)
        

## Face Recogition from Webcam

In [ ]:
from __future__ import print_function
from facenet_pytorch import MTCNN, InceptionResnetV1,extract_face
from PIL import Image,ImageDraw
import torch
import cv2
import os
import torch.nn as nn

from Web.imutils.video import WebcamVideoStream
from Web.imutils.video import FPS
import imutils



size=4
i=1
classifier = cv2.CascadeClassifier('/home/mj/anaconda3/share/opencv4/haarcascades/haarcascade_frontalface_default.xml')
def cos_sim(a, b):
    """Takes 2 vectors a, b and returns the cosine similarity according 
    to the definition of the dot product"""
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

def cos(a,b):
    minx = -1 
    maxx = 1
    return (cos_sim(a,b)- minx)/(maxx-minx)


def verify(embedding):

    for i,k in enumerate(embeddings):
        for j,l in enumerate(embedding):
            dist =cos(k,l)
    
        #print(dist)
    # Chosen threshold is 0.7  
            if dist > 0.85:
                text= names[i]
                cv2.putText(ima, text,(boxes[j][0].astype(int),boxes[j][3].astype(int)), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (0,255,0), 2)
                print(text)
        
                
device = torch.device('cpu')        
resnet = InceptionResnetV1(pretrained='vggface2').eval().to('cpu')
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, prewhiten=True,
    device=device,keep_all=True
)

vs = WebcamVideoStream(src=0).start()
print("camera open")
while True:
    im= vs.read()
    im=cv2.flip(im,1) #Flip to act as a mirror
    
    try:
        frame = imutils.resize(im, width=400)
        #gray=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        # Resize the image to speed up detection

       # detect MultiScale / faces 
        faces = classifier.detectMultiScale(frame)
        path="./data/testing...".format(i)
        if not os.path.exists('./data/testing...'.format(i)):
            os.makedirs('./data/testing...'.format(i))
        img_name = "im_{}.jpg".format(i)    
        cv2.imwrite(os.path.join(path,img_name),frame)


        imgName="./data/testing.../im_{}.jpg".format(i)
        #print("here:",imgName)
    # Get cropped and prewhitened image tensor
        img=Image.open(imgName)
        #print("there:",imgName)
        i=i+1
        #print(img)

        img_cropped = mtcnn(img)
        boxes,prob=mtcnn.detect(img)
        img_draw = img.copy()
        draw = ImageDraw.Draw(img_draw)
        for i, box in enumerate(boxes):
            draw.rectangle(box.tolist())
            extract_face(img, box, save_path='detected_face_{}.png'.format(i))
        img_draw.save('annotated_faces.png')
        ima=cv2.imread('annotated_faces.png')
        #print(img_cropped)

    # Calculate embedding (unsqueeze to add batch dimension)
        img_embedding = resnet(img_cropped)
       # print(img_embedding)

        cos_sim = nn.CosineSimilarity(dim=-1, eps=1e-6)
        verify(img_embedding)
        cv2.imshow('Detecting...',ima)
    
    except:
        text="No image found"
        cv2.putText(ima, text, (((box[2]-box[0])/2).astype(int),box[3].astype(int)), cv2.FONT_HERSHEY_SCRIPT_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Detecting...',ima)
            
    
       
    key = cv2.waitKey(1)
    
        # if Esc key is press then break out of the loop 
    if key == 27:#The Esc key
        break         
cv2.destroyAllWindows() 
vs.stop()
#print(img_embedding.size())
# Or, if using for VGGFace2 classification
##resnet.classify = True
#img_probs = resnet(img_cropped.unsqueeze(0))
#print(img_cropped)
#print(img_probs)